In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse

In [330]:
from sklearn.model_selection import train_test_split

In [331]:
data = pd.read_csv('input/train.csv')

In [332]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [333]:
data = data.loc[data['Rooms']<10,:]

data = data.loc[data['Price'].between(30000,600000),:]

data['LifeSquare'] = data['LifeSquare'].fillna(data['Square'])

data.loc[(data['Square']<15)&(data['LifeSquare']>15),'Square'] = data['LifeSquare']

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']<=1),'Square'] = data.loc[data['Rooms']==1,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==2),'Square'] = data.loc[data['Rooms']==2,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==3),'Square'] = data.loc[data['Rooms']==3,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==4),'Square'] = data.loc[data['Rooms']==4,'Square'].mean()

data.loc[(data['Square']<15)&(data['LifeSquare']<15)&(data['Rooms']==5),'Square'] = data.loc[data['Rooms']==5,'Square'].mean()

data.loc[(data['Square']>15)&(data['LifeSquare']<15),'LifeSquare'] = data['Square']

data.loc[data['Square']<data['LifeSquare'],'LifeSquare'] = data['Square']

data.loc[data['HouseYear']>2019,'HouseYear'] = data.loc[:,'HouseYear'].mean()

data.loc[data['KitchenSquare']>data['Square'],'KitchenSquare'] = data.loc[data['Rooms']==1,'KitchenSquare'].mean()

data.loc[data['Floor']>data['HouseFloor'],'HouseFloor'] = data['Floor']

data = pd.get_dummies(data)

data.columns

Index(['Id', 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Social_1', 'Social_2',
       'Social_3', 'Healthcare_1', 'Helthcare_2', 'Shops_1', 'Price',
       'Ecology_2_A', 'Ecology_2_B', 'Ecology_3_A', 'Ecology_3_B', 'Shops_2_A',
       'Shops_2_B'],
      dtype='object')

In [334]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9977 entries, 0 to 9999
Data columns (total 23 columns):
Id               9977 non-null int64
DistrictId       9977 non-null int64
Rooms            9977 non-null float64
Square           9977 non-null float64
LifeSquare       9977 non-null float64
KitchenSquare    9977 non-null float64
Floor            9977 non-null int64
HouseFloor       9977 non-null float64
HouseYear        9977 non-null float64
Ecology_1        9977 non-null float64
Social_1         9977 non-null int64
Social_2         9977 non-null int64
Social_3         9977 non-null int64
Healthcare_1     5185 non-null float64
Helthcare_2      9977 non-null int64
Shops_1          9977 non-null int64
Price            9977 non-null float64
Ecology_2_A      9977 non-null uint8
Ecology_2_B      9977 non-null uint8
Ecology_3_A      9977 non-null uint8
Ecology_3_B      9977 non-null uint8
Shops_2_A        9977 non-null uint8
Shops_2_B        9977 non-null uint8
dtypes: float64(9), int6

In [337]:
train, valid = train_test_split(data, test_size=0.3, random_state=42)

In [338]:
fts = ['Square','Rooms','LifeSquare','Ecology_1','Social_3','Helthcare_2','Shops_1','Ecology_2_A','Ecology_2_B','Ecology_3_A','Ecology_3_B','Shops_2_A','Shops_2_B','HouseYear','Floor','HouseFloor','KitchenSquare']

линейная регрессия

In [339]:
lr = LinearRegression()

In [340]:
lr.fit(train.loc[:,fts], train['Price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [341]:
pred = lr.predict(train.loc[:,fts])

In [342]:
pred

array([294072.36816661, 299040.92179837, 262224.71247431, ...,
       223616.82682327, 156386.93894505, 119152.35184776])

In [343]:
r2(train['Price'], pred)

0.4770074733003947

In [344]:
pred_valid = lr.predict(valid.loc[:,fts])

In [345]:
r2(valid['Price'],pred_valid)

0.48003249843255547

RF

In [346]:
from sklearn.ensemble import RandomForestRegressor as RF

In [347]:
clf = RF(max_depth=18,n_estimators=40,max_features=3,random_state=100)

In [348]:
parameters = [{'n_estimators': [20,25,30,35,40,45,50],'max_features': np.arange(3,5),'max_depth': np.arange(10,30)}]

In [349]:
from sklearn.model_selection import GridSearchCV

In [350]:
clf = GridSearchCV(estimator=RF(random_state=100),param_grid=parameters,scoring='r2',cv=3)

In [351]:
clf.fit(train.loc[:,fts],train['Price'])

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=100, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [20, 25, 30, 35, 40, 45, 50], 'max_features': array([3, 4]), 'max_depth': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [352]:
clf.best_params_

{'max_depth': 26, 'max_features': 4, 'n_estimators': 50}

In [353]:
pred = clf.predict(train.loc[:,fts])

In [354]:
r2(train['Price'], pred)

0.9605050421973659

In [355]:
pred_valid = clf.predict(valid.loc[:,fts])

In [356]:
r2(valid['Price'],pred_valid)

0.7329242999726164

In [357]:
from sklearn.metrics import mean_squared_error as MSE

In [358]:
MSE(valid['Price'],pred_valid)

2117030178.2554219

дерево решений

In [359]:
from sklearn.tree import DecisionTreeRegressor

In [360]:
regressor = DecisionTreeRegressor(max_depth=21, random_state=100)

In [361]:
regressor.fit(train.loc[:,fts],train['Price'])

DecisionTreeRegressor(criterion='mse', max_depth=21, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=100, splitter='best')

In [362]:
pred = regressor.predict(train.loc[:,fts])

In [363]:
r2(train['Price'], pred)

0.9929516641890029

In [364]:
pred_valid = regressor.predict(valid.loc[:,fts])

In [365]:
r2(valid['Price'],pred_valid)

0.4876303457222443

KNN

In [366]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor as KNN

масштабирование

In [367]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train.loc[:,fts])
valid_scaled = scaler.transform(valid.loc[:,fts])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [368]:
for i in range(2,12,2):
    print(f'n_neighbors={i}')
    knn = KNN(n_neighbors=i,weights='distance')
    knn.fit(train_scaled, train['Price'])
    pred = knn.predict(train_scaled)
    print(r2(train['Price'],pred))
    pred_valid = knn.predict(valid_scaled)
    print(r2(valid['Price'],pred_valid))
    print()

n_neighbors=2
1.0
0.5854174231055721

n_neighbors=4
1.0
0.6307859042820455

n_neighbors=6
1.0
0.6419533623871729

n_neighbors=8
1.0
0.6448303824288586

n_neighbors=10
1.0
0.6466691261262059



Вывод: 
    самая лучшая модель Random Forest (случайный лес). 
    Она дает наилучшие предсказания и ее будем использовать для теста!

In [369]:
test = pd.read_csv('input/test.csv')

In [370]:
test.head(5000)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A
5,7633,53,1.0,40.675627,NaN,1.0,21,21.0,1977,0.049637,B,B,34,7759,0,229.0,1,3,B
6,13329,23,2.0,68.099538,64.843025,1.0,2,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
7,5502,32,2.0,48.193897,32.857644,6.0,5,14.0,1972,0.135650,B,B,46,7960,6,350.0,3,11,B
8,4220,96,3.0,72.277106,45.968758,9.0,17,17.0,1997,0.041116,B,B,53,14892,4,NaN,1,4,B
9,11538,6,3.0,80.219400,47.660260,9.0,13,17.0,2014,0.243205,B,B,5,1564,0,540.0,0,0,B


In [371]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [372]:
test['LifeSquare'] = test['LifeSquare'].fillna(test['Square'])

test.loc[(test['Square']<15)&(test['LifeSquare']>15),'Square'] = test['LifeSquare']

test.loc[(test['Square']<15)&(test['LifeSquare']<15)&(test['Rooms']<=1),'Square'] = test.loc[test['Rooms']==1,'Square'].mean()

test.loc[(test['Square']<15)&(test['LifeSquare']<15)&(test['Rooms']==2),'Square'] = test.loc[test['Rooms']==2,'Square'].mean()

test.loc[(test['Square']<15)&(test['LifeSquare']<15)&(test['Rooms']==3),'Square'] = test.loc[test['Rooms']==3,'Square'].mean()

test.loc[(test['Square']<15)&(test['LifeSquare']<15)&(test['Rooms']==4),'Square'] = test.loc[test['Rooms']==4,'Square'].mean()

test.loc[(test['Square']<15)&(test['LifeSquare']<15)&(test['Rooms']==5),'Square'] = test.loc[test['Rooms']==5,'Square'].mean()

test.loc[(test['Square']>15)&(test['LifeSquare']<15),'LifeSquare'] = test['Square']

test.loc[test['Square']<test['LifeSquare'],'LifeSquare'] = test['Square']

test.loc[test['HouseYear']>2019,'HouseYear'] = test.loc[:,'HouseYear'].mean()

test.loc[test['KitchenSquare']>test['Square'],'KitchenSquare'] = test.loc[test['Rooms']==1,'KitchenSquare'].mean()

test.loc[test['Floor']>test['HouseFloor'],'HouseFloor'] = test['Floor']

test = pd.get_dummies(test)


In [373]:
pred_test = clf.predict(test.loc[:,fts])

In [374]:
test['Price'] = pred_test

In [375]:
test.loc[:,['Id','Price']]

,Id,Price
0,725,168749.017226
1,15856,217029.331076
2,5480,208763.348693
3,15664,331033.044486
4,14275,137482.490319
5,7633,213804.089555
6,13329,176226.791361
7,5502,214374.535997
8,4220,307475.791875
9,11538,211366.197441


In [377]:
test.loc[:,['Id','Price']].to_csv('output/VZinchenko_predictions.csv',index=None)